# Loco Mosquito

The program to extract, clean and prepare the IR spectra from mosquitoes. 



### Changelog

    1.0 - Original program
    1.1 - Adapted for new file names with country code
        - Added code to avoid processing files with wrong names
        - Solved bug in the code that plots a spectrum and the selected wavenumbers
    2.0 - Added an algorithm to being able to read files with names without country code
        - It plots a bar and wiskers graph with the results of the mosquitoes.
        - Added an algorithm to detect when the selected wavenumbers are out of the measured ranges
    3.0 - Supports .mzz files
        



### Useful packages

First of all, we are going to load all the packages that we will need for our program

In [ ]:
import numpy as np #we will use numpy to process the data
import os 
import time
import csv
import matplotlib.pyplot as plt
import zipfile
import zlib

### Loading the spectra

All spectra to be processed must be stored in a text file, in the same folder and with a descriptive name of their status, which must follow the following format:

```
kk-C-xxD-yy-888888-555555-zz.dpt
```

where:

* **kk**: Species code
    * **AR**: _Anopheles arabiensis_
    * **AG**: _Anopheles gambiae_
    * **AC**: _Anopheles coluzzi_
* **C**: Country code
    * **B**: Burkina Faso
    * **T**: Tanzania
    * **S**: Scotland
* **xx**: Age of the mosquito in days
* **yy**: Status of the mosquito
    * **BF**: Blood fed
    * **SF**: Sugar fed
    * **UF**: Unfed
    * **GR**: Gravid
* **888888**: Date the mosquito was measured
* **555555**: Date the mosquito was collected
* **zz**: Mosquito number identifier

Although it is also possible to read files with the old naming system without country code:

```
kk-xxD-yy-888888-555555-zz.dpt
```

Here is the algorithm to load the spectra, but first you must indicate the folder location. To do this, the easiest way to proceed is just going to a file in the folder where the spectra are, right-click on it, select properties, and, then, copy the text at the right of "Location:". Remember to add `\` at the end if you are using Windows or `/` if you are using an UNIX os

In [ ]:
spectra_path = input("Please, indicate the folder location: ")

In [ ]:
a = time.time()
# find all the .mzz and .dpt files in the folder and dicard the files with wrong names
dptfiles = [f for f in os.listdir(spectra_path) if f[-3:] == "dpt"]
mzzfiles = [f for f in os.listdir(spectra_path) if f[-3:] == "mzz"]
if len(mzzfiles) > 0:
    spectra_names = mzzfiles
    mzzq = True
else:
    spectra_names = dptfiles
    mzzq = False
matrix = []
list_ages = []
list_status = []
list_species = []
# Now we load the spectra in a matrix
for i in spectra_names:
    # This is the code to avoid errors with files using different naming systems
    if len(i.split(".")[0]) == 25:
        pl = 2
    elif len(i.split(".")[0]) == 23:
        pl = 0
    else:
        continue
    # Now we read the species from the name
    species = i[:2]
    if species in list_species:
        pass
    else:
        list_species.append(species)
    # Then the age (from here everything depends on the naming system)
    age = i[3+pl:5+pl]
    if age in list_ages:
        pass
    else:
        list_ages.append(age)
    # Then the status:
    status = i[7+pl:9+pl]
    if status in list_status:
        pass
    else:
        list_status.append(status)
    # Then the time that the mosquitoes where stored
    colday = time.mktime(time.strptime(i[10+pl:16+pl],"%y%m%d"))
    mesday = time.mktime(time.strptime(i[17+pl:23+pl],"%y%m%d"))
    stime = abs((mesday - colday) / (3600 * 24))
    # And finally the spectrum and its characteristics
    if mzzq == False:
        with open(spectra_path + "/" + i, 'rb') as tmp:
            avmi = (line.replace(b'\t',b',') for line in tmp)
            spectrum = np.genfromtxt(avmi, delimiter=',')
        start = spectrum[0,0]
        end = spectrum[-1,0]
        ls = len(spectrum)
        spectrum = np.transpose(spectrum)[1]    
    else:
        with zipfile.ZipFile(spectra_path + "/" + i) as myzip:
            with myzip.open(i[:-4] + ".tmp") as myfile:
                avmi = (line.replace(b'\t',b',') for line in myfile)
                spectrum = np.genfromtxt(avmi, delimiter=',')
        start = spectrum[0]
        end = spectrum[1]
        ls = int(spectrum[2])
        spectrum = spectrum[3:]
    matrix.append([species,age,status,stime,[start,end,ls],spectrum])
list_species = sorted(list_species)
list_ages = sorted(list_ages)
list_status = sorted(list_status)
b = time.time()
print("This last process has lasted " + str(round(b-a,3)) + " s.")

The following code allows us to count the number of mosquitoes for each specie, age, and status. 

In [ ]:
mos_acc = np.zeros((len(list_species),len(list_status),len(list_ages)),dtype=int) # We create an empty table for the 
for i in range(len(matrix)):
    x = list_species.index(matrix[i][0])
    y = list_status.index(matrix[i][2])
    z = list_ages.index(matrix[i][1])
    mos_acc[x][y][z] += 1 
    
# We show it in a beautiful way
width = ((len(list_ages) + 1) * 5) - 2
print()
for i in range(mos_acc.shape[0]):
    print(" " * (round(width/2) -2) + list_species[i])
    print("=" * width)
    print("    "+'   '.join(map(str, list_ages))) 
    print("-" * width)
    for j in range(mos_acc.shape[1]):
        print(list_status[j] + " " + "  ".join(["%3.0f" % dx for dx in mos_acc[i][j]]))
    print()

## Detection of bad spectra

### Spectra with low intensity

If the mosquito was not well placed at the ATR's crystal, the intensity of the whole spectrum is small. Our experience says that we can use as reference the small plateau between 400 and 500 wavenumbers that the mosquito spectra usually have. Since, the spectrometer with ZnSe optics only can reach to 500 wavenumbers, it doesn matter if we extend this range to 600 cm<sup>-1</sup>. Then if the average of this reference is smaller than 0.11, the spectrum doesn't have enough quality to be scaled and, then, of course, employed.

In [ ]:
# A list of the discarted spectra will be collected:
bad_spectra = []
for i in range(len(matrix)):
    # first we calculate the position of the points that comprise that section of the spectrum
    if matrix[i][4][1] < 600 and matrix[i][4][1] > 400:
        sta = int(round((((matrix[i][4][2] - 1) / (matrix[i][4][1] - matrix[i][4][0])) * (600 - matrix[i][4][0])) + 1)) - 1
        end = matrix[i][4][2]
    elif matrix[i][4][1] <= 400:
        sta = int(round((((matrix[i][4][2] - 1) / (matrix[i][4][1] - matrix[i][4][0])) * (600 - matrix[i][4][0])) + 1)) - 1
        end = int(round((((matrix[i][4][2] - 1) / (matrix[i][4][1] - matrix[i][4][0])) * (400 - matrix[i][4][0])) + 1)) - 1
    else:
        sta = 0 # if the spectrum doesn't reach 600 cm-1 we cannot prove if the spectrum has enough intensity
        raise Exception("The spectrum {} doesn't reach 600 cm-1".format(spectra_names[1]))
    # Now we check the intensity of the spectra in that region. If is not over 0.1 we discard the spectrum
    if np.average(matrix[i][5][sta:end]) < 0.11:
        bad_spectra.append("LI: " + spectra_names[i])
        matrix[i] = None
if (bad_spectra) == 1:
    print("1 spectrum has been discarded because its low intensity")
else:
    print(str(len(bad_spectra)) + " spectra have been discarded because their low intensity")

### Spectra with atmospheric interferences

If the spectra were measured after the change of the beamspliter or after installing the ATR and the background was not correctly measured, the spectra will be with the interference of the water and CO<sub>2</sub> spectra. In the case of water vapour, its IR spectrum has three bands very noisy: one between 4000 and 3400 cm<sup>-1</sup>, other between 2200 and 1300 cm<sup>-1</sup> and the last one starts to appear below 800 cm<sup>-1</sup>. Because the second band appears exactly were the most interesting bands of the mosquitoes are, we are going to use the first band to detect the spectra contaminated by the water vapor and remove it from our data. In order to do that we check how smooth are the spectra between 3900 and 3500 cm<sup>-1</sup>, fitting that region to a polynomial. Only the spectra without noise will fit well.

In [ ]:
bs = 0 # counter for the number of spectra discarderd

# Now we define a function to calculate the R-squared coefficient of the fitting of our data to a polynomial
def rs_pf(x, y, degree):
    coeffs = np.polyfit(x, y, degree)
    # r-squared
    p = np.poly1d(coeffs)
    # fit values, and mean
    yhat = p(x)                         # or [p(z) for z in x]
    ybar = np.sum(y)/len(y)          # or sum(y)/len(y)
    ssreg = np.sum((yhat-ybar)**2)   # or sum([ (yihat - ybar)**2 for yihat in yhat])
    sstot = np.sum((y - ybar)**2)    # or sum([ (yi - ybar)**2 for yi in y])
    results = ssreg / sstot

    return results

# Here take that the section of the data between 3900 and 3500 cm-1 and check if it fits well to a 5th degree polinomial
for i in range(len(matrix)):
    if matrix[i]: #to check if we have spectra
        # Now one would spect that the spectrum will reach 3900 so the program will not check it out.
        sta = int(round((((matrix[i][4][2] - 1) / (matrix[i][4][1] - matrix[i][4][0])) * (3900 - matrix[i][4][0])) + 1)) - 1
        end = int(round((((matrix[i][4][2] - 1) / (matrix[i][4][1] - matrix[i][4][0])) * (3500 - matrix[i][4][0])) + 1)) - 1
        # we take that data:
        yd = matrix[i][5][sta:end]
        xd = list(range(len(yd)))
        rs = rs_pf(xd,yd,5)
        # And now, if the fitting is bad, we discard the spectrum
        if rs < 0.99:
            bs +=1
            bad_spectra.append("AI: " + spectra_names[i])
            matrix[i] = None
if (bs) == 1:
    print("1 spectrum has been discarded because has atmospheric interferences")
else:
    print(str(bs) + " spectra have been discarded because have atmospheric interferences")   

### Spectra shifted by the anvil

Sometimes, when you are measuring a mosquito, the mosquito moves out of the ATR crystal and what you get is a plain spectrum without features (apart from CO<sub>2</sub> bands) but with high intensity. Sometimes these spectra pass the previous filters and it is necessary to remove them. To do it that we are going to do is to select the wavenumber with less signal from the mosquito (this wavenumber is usually 1900 cm<sup>-1</sup>) and look for outliers at that frequency.

In [ ]:
bs = 0 # counter for the number of spectra discarderd 
# we calculate the fences of the data set based in a value we can choose (in statistics 1.5 times
# the interquartile range is the inner fence and 3 time is the outer fence)
l = 1.5
# We look for the point at 1900 cm-1 and add it to the list of intensities
li = []
for i in range(len(matrix)):
    if matrix[i]: #to check if we have spectra
        # Now one would spect that the spectrum will reach 3900 so the program will not check it out.
        sta = int(round((((matrix[i][4][2] - 1) / (matrix[i][4][1] - matrix[i][4][0])) * (1900 - matrix[i][4][0])) + 1)) - 1
        li.append(matrix[i][5][sta])
q3, q1 = np.percentile(li, [75 ,25])
ir = q3 - q1
for i in range(len(matrix)):
    if matrix[i]: #to check if we have spectra
        sta = int(round((((matrix[i][4][2] - 1) / (matrix[i][4][1] - matrix[i][4][0])) * (1900 - matrix[i][4][0])) + 1)) - 1
        if matrix[i][5][sta] > (l * ir + q3) or matrix[i][5][sta] < (q1 - l * ir):
            bs +=1
            bad_spectra.append("SA: " + spectra_names[i])
            matrix[i] = None 
if (bs) == 1:
    print("1 spectrum has been discarded because it was distorted by the anvil")
else:
    print(str(bs) + " spectra have been discarded because they were distorted by the anvil")  

### Number of mosquitoes after the filtering

In [ ]:
mos_acc = np.zeros((len(list_species),len(list_status),len(list_ages)),dtype=int) # We create an empty table for the 
for i in range(len(matrix)):
    if matrix[i]:
        x = list_species.index(matrix[i][0])
        y = list_status.index(matrix[i][2])
        z = list_ages.index(matrix[i][1])
        mos_acc[x][y][z] += 1 
    
# We show it in a beautiful way
width = ((len(list_ages) + 1) * 5) - 2
print()
for i in range(mos_acc.shape[0]):
    print(" " * (round(width/2) -2) + list_species[i])
    print("=" * width)
    print("    "+'   '.join(map(str, list_ages))) 
    print("-" * width)
    for j in range(mos_acc.shape[1]):
        print(list_status[j] + " " + "  ".join(["%3.0f" % dx for dx in mos_acc[i][j]]))
    print()

## Selection of the wavenumbers and data extraction

Now, we proceed to finish the task extracting the intensity of the remaining spectra at the wavenumbers that we want. We just need to indicate those wavenumbers at the next list:

In [ ]:
wns = [3855, 3400, 3275, 2922, 2853, 1900, 1745, 1635, 1539, 1457, 1306, 1154, 1076, 1027, 880, 525, 401]

We can see in a spectrum the wavenumbers selected

In [ ]:
# We select a random spectrum:
n = np.random.randint(0,len(matrix)-1)
while not matrix[n]:
    n = np.random.randint(0,len(matrix)-1)

# we prepare the data
a = matrix[n][4][0]
b = matrix[n][4][1]
c = matrix[n][4][2]
xd = [a - x/c * (a-b) for x in range(c)]
yd = matrix[n][5]

# we draw the plot
plt.figure(figsize=(14,7))
plt.plot(xd,yd)
plt.xlim(a, b)

# and the selected wavenumbers
for i in wns:
    plt.axvline(x=i, c='black', lw=1)
    
plt.xlabel('Wavenumber')
plt.ylabel('Absorbance')
plt.title('Selected Wavenumbers')

plt.show()

And now, the following algorithm will extract from the spectra the desired intensities:

In [ ]:
# We start the timer
a = time.time()
# We define the variable that will contain the final data
fida = []
csc = 0
# And start the algorithm to extract the info
for i in matrix:
    # If that item exist
    if i:
        # we count the number of spectra that are removed because are too short for the range of wavenumbers selected
        if i[4][0] > wns[0] and i[4][1] < wns[-1]:
            pos = []
            for j in wns:
                pos.append(int(round((((i[4][2] - 1) / (i[4][1] - i[4][0])) * (j - i[4][0])) + 1)) - 1)
            lint = []
            for k in pos:
                lint.append(i[5][k])
            fida.append([i[0], i[1], i[2],str(int(i[3]))] + lint)
            fida = sorted(fida)
        else:
            csc += 1
fida.insert(0,["Species","Age","Status","StoTime"] + wns)
if (csc) == 0:
    pass
elif (csc) == 1:
    print("1 spectrum has been discarded because was shorter than the selected wavenumbers")
else:
    print(str(csc) + " spectra have been discarded because were shorter than the selected wavenumbers")  
b = time.time()
print("This last process has lasted " + str(round(b-a,3)) + " s.")

## Saving the matrix

Now we export the matrix with the info in the same folder where the data were collected

In [ ]:
with open(spectra_path + "mosquitos.dat", 'w') as file:
    sc = csv.writer(file, delimiter='\t')
    for i in fida:
            sc.writerow(i)

## Optional: Plotting the results

First, we select the data that we want to include in the plot:

In [ ]:
sel_spec = ["KS","AR"]
sel_ages = ["01","03","05","07","09","11","13","15","17"]
sel_stat = ["BF","GR","SF"]

In [ ]:
# Create a matrix with zeros
bpd = [[[0] for x in range(len(sel_ages))] for y in range(len(wns))]
# Then, we add the values for each wavenumber, each age:
for i in fida[1:]: #to skip the first row with the info
    if i[0] in sel_spec and i[1] in sel_ages and i[2] in sel_stat:
        for j, k in enumerate(i[-len(wns):]):
            bpd[j][sel_ages.index(i[1])].append(k)
# we remove the zeros
for k, i in enumerate(bpd):
    for l, j in enumerate(i):
        if j[0] == 0:
            bpd[k][l] = bpd[k][l][1:]

Now we plot the figure:

In [ ]:
a = [i + 1 for i in range(len(sel_ages))]
plt.figure(figsize=(18, 30))
for i in range(len(wns)):
    ax1 = plt.subplot(len(wns) // 3 + len(wns) % 3, 3, i+1)
    plt.title(str(wns[i]) + " cm$^{-1}$")
    ax1.boxplot(bpd[i])
    ax1.set_xticks(a)
    ax1.set_xticklabels(sel_ages, minor=False)
# plt.savefig("C:\\Users\\Mario\\Google Drive\\160822 - Mosquitos\\whiskers.png", dpi = 150)
plt.show()
